<a href="https://colab.research.google.com/github/ishapugalia/Abstractive-Text-Summarization/blob/main/model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import os
import pandas as pd
import re

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [3]:
import tensorflow as tf

In [6]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dataset csv/train.csv")


In [5]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
d_articles=pd.DataFrame(data.iloc[:10000,[0,1]])
print(d_articles[:1])
d_highlights=pd.DataFrame(data.iloc[:10000,[0,2]])
print(d_highlights.columns)
#print(d_articles)

                                         id  \
0  0001d1afc246a7964130f43ae940af6bc6c57f01   

                                             article  
0  By . Associated Press . PUBLISHED: . 14:11 EST...  
Index(['id', 'highlights'], dtype='object')


In [31]:
num_words = 30000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'

In [8]:
def cleantext(text):
    text = str(text)
    text=text.split()
    words=[]
    for t in text:
        if t.isalpha():
            words.append(t)
    text=" ".join(words)
    text=text.lower()
    text=re.sub(r"what's","what is ",text)
    text=re.sub(r"it's","it is ",text)
    text=re.sub(r"\'ve"," have ",text)
    text=re.sub(r"i'm","i am ",text)
    text=re.sub(r"\'re"," are ",text)
    text=re.sub(r"n't"," not ",text)
    text=re.sub(r"\'d"," would ",text)
    text=re.sub(r"\'s","s",text)
    text=re.sub(r"\'ll"," will ",text)
    text=re.sub(r"can't"," cannot ",text)
    text=re.sub(r" e g "," eg ",text)
    text=re.sub(r"e-mail","email",text)
    text=re.sub(r"9\\/11"," 911 ",text)
    text=re.sub(r" u.s"," american ",text)
    text=re.sub(r" u.n"," united nations ",text)
    text=re.sub(r"\n"," ",text)
    text=re.sub(r":"," ",text)
    text=re.sub(r"-"," ",text)
    text=re.sub(r"\_"," ",text)
    text=re.sub(r"\d+"," ",text)
    text=re.sub(r"[$#@%&*!~?%{}()]"," ",text)
    
    return text

In [39]:
d_articles["article"]= d_articles["article"].apply(lambda x: cleantext(x))

In [40]:
d_highlights["highlights"]= d_highlights["highlights"].apply(lambda x: cleantext(x))

In [46]:
VOCAB_SIZE = 29999
tokenizer = Tokenizer(num_words=VOCAB_SIZE,oov_token='<UNK>')
tokenizer.fit_on_texts(d_articles.article)
article_sequences = tokenizer.texts_to_sequences(d_articles.article)
art_word_index = tokenizer.word_index


75953

In [50]:
print(art_word_index.items())

dict_items([('<UNK>', 1), ('the', 2), ('to', 3), ('a', 4), ('and', 5), ('of', 6), ('in', 7), ('was', 8), ('for', 9), ('that', 10), ('on', 11), ('is', 12), ('he', 13), ('with', 14), ('his', 15), ('as', 16), ('at', 17), ('it', 18), ('by', 19), ('have', 20), ('from', 21), ('has', 22), ('be', 23), ('her', 24), ('but', 25), ('are', 26), ('said', 27), ('an', 28), ('she', 29), ('they', 30), ('who', 31), ('had', 32), ('not', 33), ('i', 34), ('their', 35), ('been', 36), ('were', 37), ('this', 38), ('after', 39), ('will', 40), ('when', 41), ('which', 42), ('one', 43), ('about', 44), ('we', 45), ('more', 46), ('or', 47), ('up', 48), ('out', 49), ('would', 50), ('also', 51), ('you', 52), ('new', 53), ('two', 54), ('all', 55), ('into', 56), ('last', 57), ('people', 58), ('told', 59), ('than', 60), ('can', 61), ('just', 62), ('so', 63), ('there', 64), ('over', 65), ('first', 66), ('its', 67), ('what', 68), ('if', 69), ('police', 70), ('could', 71), ('him', 72), ('my', 73), ('mr', 74), ('no', 75), ('

In [47]:
art_vocab = {}
counter = 0
for word in art_word_index.keys():
    if art_word_index[word] == 0:
        print("found 0!")
        break
    if art_word_index[word] > VOCAB_SIZE:
        continue
    else:
        art_vocab[word] = art_word_index[word]
        counter += 1

In [51]:
print(art_vocab)

{'<UNK>': 1, 'the': 2, 'to': 3, 'a': 4, 'and': 5, 'of': 6, 'in': 7, 'was': 8, 'for': 9, 'that': 10, 'on': 11, 'is': 12, 'he': 13, 'with': 14, 'his': 15, 'as': 16, 'at': 17, 'it': 18, 'by': 19, 'have': 20, 'from': 21, 'has': 22, 'be': 23, 'her': 24, 'but': 25, 'are': 26, 'said': 27, 'an': 28, 'she': 29, 'they': 30, 'who': 31, 'had': 32, 'not': 33, 'i': 34, 'their': 35, 'been': 36, 'were': 37, 'this': 38, 'after': 39, 'will': 40, 'when': 41, 'which': 42, 'one': 43, 'about': 44, 'we': 45, 'more': 46, 'or': 47, 'up': 48, 'out': 49, 'would': 50, 'also': 51, 'you': 52, 'new': 53, 'two': 54, 'all': 55, 'into': 56, 'last': 57, 'people': 58, 'told': 59, 'than': 60, 'can': 61, 'just': 62, 'so': 63, 'there': 64, 'over': 65, 'first': 66, 'its': 67, 'what': 68, 'if': 69, 'police': 70, 'could': 71, 'him': 72, 'my': 73, 'mr': 74, 'no': 75, 'being': 76, 'some': 77, 'other': 78, 'before': 79, 'time': 80, 'like': 81, 'because': 82, 'where': 83, 'while': 84, 'them': 85, 'now': 86, 'only': 87, 'years': 88

In [44]:
tokenizer.fit_on_texts(d_highlights)
summary_sequences = tokenizer.texts_to_sequences(d_highlights.highlights)
sum_word_index = tokenizer.word_index
len(sum_word_index)

75953

In [45]:
sum_vocab = {}
counter = 0
for word in sum_word_index.keys():
    if sum_word_index[word] == 0:
        print("found 0!")
        break
    if sum_word_index[word] > VOCAB_SIZE:
        continue
    else:
        sum_vocab[word] = sum_word_index[word]
        counter += 1

In [52]:
MAX_LEN = 400
pad_article = pad_sequences(article_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [53]:
MAX_LEN_S= 100
pad_highlight = pad_sequences(summary_sequences, maxlen=MAX_LEN_S, padding='post', truncating='post')

In [54]:
pad_article.shape
pad_highlight

array([[ 4165,   287,     6, ...,     0,     0,     0],
       [  773,  6956,   153, ...,     0,     0,     0],
       [ 2726,    32,  3609, ...,     0,     0,     0],
       ...,
       [ 7178,     8,   962, ...,     0,     0,     0],
       [ 1371,    12,  2123, ...,     0,     0,     0],
       [23614, 21279,     8, ...,     0,     0,     0]], dtype=int32)

In [55]:
embeddings_index = {}
with open('/content/drive/MyDrive/Colab Notebooks/glove.6B.200d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

In [56]:
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [57]:
art_embedding_matrix = embedding_matrix_creater(200, word_index=art_vocab)
art_embedding_matrix.shape

(30000, 200)

In [59]:
df=pd.DataFrame(art_embedding_matrix)
df.to_excel("/content/drive/MyDrive/Colab Notebooks/trained/art_embedding_matrix.xlsx")


In [60]:
import pickle

In [ ]:
filehandler = open("/content/drive/MyDrive/Colab Notebooks/trained/art_vocab", 'wb')
pickle.dump(art_vocab, filehandler)

In [ ]:
filehandler = open("/content/drive/MyDrive/Colab Notebooks/trained/sum_vocab", 'wb')
pickle.dump(sum_vocab, filehandler)

In [ ]:
#file = open('/content/drive/MyDrive/Colab Notebooks/trained/art_vocab.txt', 'rb')

# dump information to that file
#data1 = pickle.load(file)
#print(data1)

In [61]:
sum_embedding_matrix = embedding_matrix_creater(200, word_index=sum_vocab)
sum_embedding_matrix.shape

(30000, 200)

In [62]:
art_embedding_matrix[10]

array([ 1.48049995e-01,  1.08750001e-01, -3.62779982e-02, -1.53860003e-01,
        3.71809989e-01,  2.93000013e-01, -7.02600002e-01,  9.56360018e-04,
        2.45509997e-01,  3.40120010e-02,  3.41380015e-02,  6.43639982e-01,
       -1.25989998e-02,  2.46610008e-02,  1.62860006e-01,  7.79170021e-02,
       -3.15250009e-01,  7.20809996e-01, -5.18419981e-01, -2.54999995e-01,
        4.89589989e-01,  3.17490005e+00, -3.79909992e-01, -3.82700004e-02,
        4.57599983e-02,  6.79299980e-02, -1.17160000e-01, -3.34100008e-01,
       -1.07230000e-01, -2.38729998e-01, -4.57599998e-01, -2.45019998e-02,
        1.47229999e-01, -1.35460004e-01,  8.98120031e-02, -3.87679994e-01,
       -9.57970023e-01, -6.20720029e-01, -1.17629997e-01,  2.13220000e-01,
        8.08940008e-02, -2.59669989e-01, -9.35840011e-02,  5.21059990e-01,
       -1.17930003e-01,  2.12239996e-01,  4.51150000e-01, -6.31339997e-02,
       -2.19540000e-01, -2.75640003e-02,  1.87400002e-02, -3.52279991e-01,
       -2.34180003e-01,  

In [63]:
def embedding_layer_creater(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, embedding_matrix):
  embedding_layer = tf.keras.layers.Embedding(input_dim = VOCAB_SIZE, 
                                    output_dim = EMBEDDING_DIM,
                                    input_length = MAX_LEN,
                                    weights = [embedding_matrix],
                                    trainable = False)
  
  return embedding_layer



In [64]:
encoder_embedding_layer = embedding_layer_creater(30000,200, MAX_LEN, art_embedding_matrix)

In [65]:
decoder_embedding_layer = embedding_layer_creater(30000,200, MAX_LEN_S, sum_embedding_matrix)

In [ ]:
if not os.path.exists("/content/drive/MyDrive/Colab Notebooks/trained"):
    os.makedirs("/content/drive/MyDrive/Colab Notebooks/trained")

In [ ]:
!pip install chart-studio

In [ ]:
from numpy.random import seed
seed(1)

from sklearn.model_selection import train_test_split
import logging

import chart_studio.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import pandas as pd
import pydot


import keras
from keras import backend as k
k.set_learning_phase(1)
from keras.preprocessing.text import Tokenizer
from keras import initializers
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Dropout,Input,Activation,Add,concatenate, Embedding, RepeatVector
from keras.layers.advanced_activations import LeakyReLU,PReLU
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [66]:
from keras.layers import TimeDistributed
import gensim

In [68]:
# Hyperparams

MAX_LEN = 400
MAX_LEN_S=100
VOCAB_SIZE=30000
EMBEDDING_DIM = 200
HIDDEN_UNITS = 256
#VOCAB_SIZE = VOCAB_SIZE + 1

LEARNING_RATE = 0.002
BATCH_SIZE = 128
EPOCHS = 10

In [69]:
from keras import backend as K 
import gensim
from numpy import *
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
import tensorflow as tf

In [78]:
from tensorflow.keras.layers import Layer
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [81]:
from keras import backend as K 
from keras.models import Sequential
K.clear_session()

latent_dim = 256
embedding_dim=200
model=Sequential()
input_len=MAX_LEN
input_l1=Input(shape=(input_len,))
# Encoder
l1=tf.keras.layers.Embedding(VOCAB_SIZE,embedding_dim,input_length=input_len,embeddings_regularizer=keras.regularizers.l2(.001))(input_l1)
outputs,s_h,s_c=LSTM(latent_dim,return_sequences=True)(l1)
cnn_layer = tf.keras.layers.Conv1D(
    filters=256,
    kernel_size=2,
    # Use 'same' padding so outputs have the same shape as inputs.
    padding='same')
attn_layer = tf.keras.layers.Attention()
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])
outputs=Dense()
print(model.summary())

(None, 256)
(None, 400)


In [77]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
filename = '/content/drive/MyDrive/Colab Notebooks/trained/model_2.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
model.fit()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


(10000, 400)
(700, 100, 30000)
10000


In [ ]:
p_art=pad_article[:700]
p_sum=pad_highlight[:700]
art_train, art_test, sum_train, sum_test = train_test_split(p_art, p_sum, test_size=0.2)


In [ ]:
train_num=art_train.shape[0]
target_train=decoder_output_data[:train_num]
target_test=decoder_output_data[train_num:]

In [ ]:
print(art_test.shape,sum_test.shape,target_test.shape,art_train.shape)

(140, 400) (140, 100) (140, 100, 30000) (560, 400)


In [ ]:
history = model.fit([art_train,sum_train], 
                     target_train, 
                     epochs=EPOCHS, 
                     batch_size=BATCH_SIZE,
                     validation_data=([art_test, sum_test], target_test))

Epoch 1/10
